#

# 1. 

# Introduction

An exploratory data analysis on a dataset of internet of Things data that represents sensor measurements recorded from interior rooms of the building, and to make suggestions to the facilities managers based on the findings.

### Question to be Answered

1. Data Quality:  Does data has any duplicate values, missing values, outliers, or errors?
2. Does data have any patterns, trends, or correlations?
3. Where is the location of sensors in the floor?
4. How does the internet of Things data vary across different sensors, and floors?
5. Is there any possible factors that affect the internet of Things data, such as outdoor weather?

### Assumptions
1. "Temperature at desk height" is considered at "Temperature" for building operation goals, which has been calculated by subtracting 1.5°C from the given Ceiling Temperature. As it is not mentioned which Temperature to take.

2. In some cases, work hours are assumed from 9 AM to 5 PM as it is also not mentioned.


### New Columns/Features 

New Columns/Features is created for data exploration

temp_c: Temperature at ceiling Hight. temp_c = temp (given temperature)

temp_d: Temperature at desk Hight. temp_d = temp_c - 1.5

Floor: Floor of the given sensor e.g. '3F'.

sensor_id_floor_lon: Combination of 'sensor_id' and 'Floor' in long format e.g. 'ENKZYW001900_4F'.

sensor_id_floor_sho: Combination of 'sensor_id' and 'Floor' in short format e.g. '1900_4F'. 

# 2. Import all the required libraries

install pandas==1.4.4

In [ ]:
import requests
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from imageio.v3 import imread
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler

## 2.1 Methods/Functions for EDA

In [ ]:
def json_api(url):

    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:

        # Parse the JSON data
        json_data = response.json()

    else:
        # If the request was unsuccessful, print an error message
        print("Failed to retrieve data from the URL. Status code:", response.status_code)
        json_data = {}
    
    return json_data

In [ ]:
def correlation(df_indoor_2,column_name):

    # Pivot the DataFrame
    pivot_df_indoor_2 = df_indoor_2.pivot_table(index='documentTime', columns='sensor_id_floor_sho', values=[column_name]).reset_index()

    # Flatten the multi-index column names
    pivot_df_indoor_2.columns = ['_'.join(col).strip() for col in pivot_df_indoor_2.columns.values]

    # Drop the 'documentTime_' column as it might not be relevant for correlation
    pivot_df_indoor_2 = pivot_df_indoor_2.drop(columns=['documentTime_'])

    # Calculate correlation matrix
    correlation_matrix = pivot_df_indoor_2.corr()
        
    return correlation_matrix

def strong_correlation_list(correlation_matrix):
    
    # Create an empty dictionary to store correlated columns
    correlated_columns = {}

    # Loop through each column in the correlation matrix
    for column in correlation_matrix.columns:
        # Get the highly correlated columns (correlation > 0.5)
        highly_correlated = correlation_matrix[column][correlation_matrix[column] > 0.5].sort_values(ascending=False)
        # Exclude the current column itself
        highly_correlated = highly_correlated.drop(column, errors='ignore')
        # Add to the dictionary
        correlated_columns[column] = highly_correlated

    # Print the correlated columns for each column
    for column, correlated_cols in correlated_columns.items():
        print(f"Highly correlated columns with '{column}':")
        print(correlated_cols)
        print()

def correlation_plot(correlation_matrix):

    # Plot heatmap using Plotly
    fig = px.imshow(correlation_matrix,
                    labels=dict(color="Correlation"),
                    x=correlation_matrix.columns,
                    y=correlation_matrix.columns,
                    title="Correlation Heatmap of Columns",
                    color_continuous_scale='RdBu')

    # Update layout to tilt x-axis labels
    fig.update_layout(xaxis={'tickangle': 45})

    # Update layout to increase height and width
    fig.update_layout(height=600, width=800)


    fig.show()

In [ ]:
def indoor_combined_single_plot(df_indoor,column_name,threhold):
    
    # Plotly line plot
    fig = go.Figure()

    # Iterate over each unique type and plot its line
    for type_val in df_indoor['sensor_id_floor_lon'].unique():
        df_type = df_indoor[(df_indoor['sensor_id_floor_lon'] == type_val)]
        fig.add_trace(go.Scatter(x=df_type['documentTime'], y=df_type[column_name], mode='lines', name=type_val))

    # Add a flat dotted line of threhold
    fig.add_trace(go.Scatter(x=[min(df_indoor['documentTime']), max(df_indoor['documentTime'])], y=[threhold, threhold],
                             mode='lines', name='Threshold line', line=dict(dash='dot', color='black', width=2)))

    if(column_name == 'temp_d'):
        title_name = 'Line Plot of Desk Temperature by Sensor'
    elif(column_name == 'humidity'):
        title_name = 'Line Plot of Humidity by Sensor'
    elif(column_name == 'co2'):
        title_name = 'Line Plot of co2 by Sensor'
        
    # Update layout
    fig.update_layout(title= title_name,
                      xaxis_title='documentTime',
                      yaxis_title= column_name)

    # Show plot
    fig.show()

def indoor_combined_different_floor_plot(df_indoor,column_name,threhold):

    # Plotly line plot
    fig = go.Figure()

    subplot_title = ['3rd Floor', '4th Floor']

    # Create subplots
    fig = make_subplots(rows=1, cols=2, subplot_titles=subplot_title)

    df_indoor_3F = df_indoor[df_indoor['Floor']=='3F']
    df_indoor_4F = df_indoor[df_indoor['Floor']=='4F']

    # Iterate over each unique type and plot its line
    for type_val in df_indoor_3F['sensor_id_floor_sho'].unique():
        df_type = df_indoor[(df_indoor['sensor_id_floor_sho'] == type_val)]
        fig.add_trace(go.Scatter(x=df_type['documentTime'], y=df_type[column_name], mode='lines', name=type_val), row=1, col=1)

    # Add a flat dotted line of threhold
    fig.add_trace(go.Scatter(x=[min(df_indoor['documentTime']), max(df_indoor['documentTime'])], y=[temp_threhold, temp_threhold],
                                 mode='lines', name='Threshold', line=dict(dash='dot', color='black', width=2)),row=1, col=1)

    # Iterate over each unique type and plot its line
    for type_val in df_indoor_4F['sensor_id_floor_sho'].unique():
        df_type = df_indoor[(df_indoor['sensor_id_floor_sho'] == type_val)]
        fig.add_trace(go.Scatter(x=df_type['documentTime'], y=df_type[column_name], mode='lines', name=type_val), row=1, col=2)

    # Add a flat dotted line of threhold
    fig.add_trace(go.Scatter(x=[min(df_indoor['documentTime']), max(df_indoor['documentTime'])], y=[threhold, threhold],
                                 mode='lines', name='Threshold', line=dict(dash='dot', color='black', width=2)),row=1, col=2)

    
    if(column_name == 'temp_d'):
        title_name = 'Line Plot of Desk Temperature by Sensor'
    elif(column_name == 'humidity'):
        title_name = 'Line Plot of Humidity by Sensor'
    elif(column_name == 'co2'):
        title_name = 'Line Plot of co2 by Sensor'
    
    # Update layout
    fig.update_layout(title=title_name,
                      xaxis_title='documentTime',
                      yaxis_title= column_name,
                     height = 500, width = 1000)

    # Show plot
    fig.show()

def sesonality(df_indoor_seasonality,column_name,duration):

    # Extracting time
    df_indoor_seasonality['time_5'] = df_indoor_seasonality['documentTime'].dt.strftime('%H:%M')

    # Extracting date
    df_indoor_seasonality['Date'] = df_indoor_seasonality['documentTime'].dt.date

    ## Group by time_5 and sensor_id_floor_lon, calculate mean
    df_mean = df_indoor_seasonality.groupby(['time_5', 'sensor_id_floor_lon']).mean().reset_index()

    ## Group by date and sensor_id_floor_lon, calculate mean
    df_mean_2 = df_indoor_seasonality.groupby(['Date', 'sensor_id_floor_lon']).mean().reset_index()

    # Create subplots
    fig = make_subplots(rows=2, cols=1, subplot_titles=("3rd Floor", "4th Floor"))

    if (duration == 'Date'):
        title_name = 'Weekly Seasonality ' + column_name
        df_rand1 = df_mean_2[df_mean_2['sensor_id_floor_lon'].str.endswith('3F')].copy()
        df_rand2 = df_mean_2[df_mean_2['sensor_id_floor_lon'].str.endswith('4F')].copy()

        # Highlighting weekends
        weekends = df_mean_2['Date'][df_mean_2['Date'].apply(lambda x: x.weekday() >= 5)].unique()

        for weekend in weekends:
            fig.add_shape(
                type="rect",
                x0=weekend,
                y0=min(df_mean_2[column_name]),
                x1=weekend + pd.Timedelta(days=1),
                y1=max(df_mean_2[column_name]),
                fillcolor="LightSkyBlue",
                opacity=0.3,
                layer="below",
                line=dict(width=0),
                row=1, col=1
            )

        for weekend in weekends:
            fig.add_shape(
                type="rect",
                x0=weekend,
                y0=min(df_mean_2[column_name]),
                x1=weekend + pd.Timedelta(days=1),
                y1=max(df_mean_2[column_name]),
                fillcolor="LightSkyBlue",
                opacity=0.3,
                layer="below",
                line=dict(width=0),
                row=2, col=1
            )


    elif(duration == 'time_5'):
        title_name = 'Daily Seasonality ' + column_name
        df_rand1 = df_mean[df_mean['sensor_id_floor_lon'].str.endswith('3F')].copy()
        df_rand2 = df_mean[df_mean['sensor_id_floor_lon'].str.endswith('4F')].copy()

    # Iterate over each unique type and plot its line
    for type_val in df_rand1['sensor_id_floor_lon'].unique():
        df_type = df_rand1[df_rand1['sensor_id_floor_lon'] == type_val]
        fig.add_trace(go.Scatter(x=df_type[duration], y=df_type[column_name], mode='lines', name=type_val, showlegend=True), row=1, col=1)


    # Iterate over each unique type and plot its line
    for type_val in df_rand2['sensor_id_floor_lon'].unique():
        df_type = df_rand2[df_rand2['sensor_id_floor_lon'] == type_val]
        fig.add_trace(go.Scatter(x=df_type[duration], y=df_type[column_name], mode='lines', name=type_val,showlegend=True), row=2, col=1)


    # Update layout
    fig.update_layout(
        title=title_name,
        xaxis_title=duration,
        yaxis_title="Mean Value",
        height = 800
    )

    # Show plot
    fig.show()


In [ ]:
def detect_outliers(df, group_column='sensor_id', based_column='temp_d', out_threshold=1.5,cond_threhold = 21, type='outlier'):
    outliers = pd.DataFrame(columns=df.columns)
    outliers1 = pd.DataFrame(columns=df.columns)
    outliers2 = pd.DataFrame(columns=df.columns)
    
    if(type=='outlier'):
      for group, data in df.groupby(group_column):
          q1 = np.percentile(data[based_column], 25)
          q3 = np.percentile(data[based_column], 75)
          iqr = q3 - q1
          lower_bound = q1 - out_threshold * iqr
          upper_bound = q3 + out_threshold * iqr
          outlier_indices = (data[based_column] < lower_bound) | (data[based_column] > upper_bound)
          #print(data[outlier_indices])
          outliers = pd.concat([outliers, data[outlier_indices]])


    elif(type=='condition'):
      for group, data in df.groupby(group_column):
        outlier_indices1 = (data[based_column] < cond_threhold)
        outlier_indices2 = (data[based_column] > cond_threhold)
        #print(data[outlier_indices])
        outliers1 = pd.concat([outliers1, data[outlier_indices1]])
        outliers2 = pd.concat([outliers2, data[outlier_indices2]])
      
      if(based_column=='co2'):
        outliers=outliers2
      else:
        outliers=outliers1

    # Extract date from rounded timestamp
    outliers['date'] = outliers['documentTime'].dt.date

    # Group by 'type_column' and date, count outliers for each sensor type for each day
    outliers_count_per_sensor_per_day = outliers.groupby(['sensor_id', 'date']).size().reset_index(name='outliers_count')
      
    return outliers,outliers_count_per_sensor_per_day


def outlier_plot(df, sensor_type):
  df = outliers_count_per_sensor_per_day

  # Create the figure object with subplots
  fig = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]], horizontal_spacing=0.2)

  # Add trace for sensor_id
  fig.add_trace(
      go.Pie(
          labels=outliers_count_per_sensor_per_day["sensor_id"],
          values=outliers_count_per_sensor_per_day["outliers_count"],
          textinfo="label+value+percent",
          name="Sensor ID",
          showlegend=False  # Show legend for this trace
      ),
      row=1, col=1
  )

  # Add trace for date
  fig.add_trace(
      go.Pie(
          labels=outliers_count_per_sensor_per_day["date"],
          values=outliers_count_per_sensor_per_day["outliers_count"],
          textinfo="label+value+percent",
          name="Date",
          showlegend=False  # Show legend for this trace
      ),
      row=1, col=2
  )

  # Add heading for the first subplot
  fig.add_annotation(
      x=0.15,
      y=-0.2,
      xref="paper",
      yref="paper",
      text="Outliers Count by Sensor ID",
      showarrow=False,
      font=dict(
          size=14,
          color="black"
      )
  )

  # Add heading for the second subplot
  fig.add_annotation(
      x=0.9,
      y=-0.2,
      xref="paper",
      yref="paper",
      text="Outliers Count by Date",
      showarrow=False,
      font=dict(
          size=14,
          color="black"
      )
  )
  title = sensor_type + " Count"
  fig.update_layout(title_text=title, height=600, width=850)
  fig.show()


In [ ]:
def find_gaps(df,duration):
    
    #Find the time difference between the consecutive dates
    df['time_diff'] = df['documentTime'].diff()
    
    if(duration=='minutes'):
    
        #Find rows with date gaps greater than 5 minutes
        gaps = df[df['time_diff'] > pd.Timedelta(minutes=15)]
    
    elif(duration=='hours'):

        #Find rows with date gaps greater than 1 hour
        gaps = df[df['time_diff'] > pd.Timedelta(hours=1)]
        
    if gaps.empty:
        print("No gaps found")
        
    else:
        print("Gaps found")
        for i in range(len(gaps)):
            gap_end = gaps.iloc[i]['documentTime']
            gap_start = df.iloc[df.index.get_loc(gaps.index[i])-1]['documentTime']
            gap_length = gaps.iloc[i]['time_diff']
            print(f"Gap start: {gap_start} | Gap end: {gap_end} | Gap length: {gap_length}")

In [ ]:
def bar_plot_overall(df,floor):
    df=df[df['Floor']==floor].copy()


    # Calculate percentage and actual count of each flag
    flag_counts = df[['Flag_Temp', 'Flag_humidity', 'Flag_co2', 'Combined_Flag']].sum()
    total_records = len(df)

    # Calculate percentage
    flag_percentages = flag_counts / total_records * 100

    # Create a list to hold both actual count and percentage for each flag
    flag_info = [(count, percentage) for count, percentage in zip(flag_counts, flag_percentages)]

    # Plot using Plotly
    fig = go.Figure()

    fig.add_trace(go.Bar(x=['Temp', 'Humidity', 'CO2', 'Combined'],
                         y=[info[1] for info in flag_info],  # Extracting percentages from flag_info list
                         text=[f"Count: {info[0]}<br>Percentage: {info[1]:.2f}%" for info in flag_info],  # Adding text for hover
                         marker_color=['blue', 'orange', 'green', 'red']))

    title_name = 'Number of points crossing given condition of period = ' + str(total_records) + " data points"
    
    fig.update_layout(title=title_name,
                      xaxis_title='Flags',
                      yaxis_title='Percentage')

    fig.show()

def incident_plot(df, display_type,floor):
    
    #sensor_type = 'temp_desk'
    group_column='sensor_id'
    #based_column='temp_d'

    incidents = pd.DataFrame(columns=df.columns)
    df=df[df['Floor']==floor].copy()

    for group, data in df.groupby(group_column):

        incidents_indices = (data['Combined_Flag'] == True)
        incidents = pd.concat([incidents, data[incidents_indices]])

    # Extract date from rounded timestamp
    incidents['date'] = incidents['documentTime'].dt.date

    # Group by 'type_column' and date, count outliers for each sensor type for each day
    outliers_count_per_sensor_per_day = incidents.groupby(['sensor_id', 'date']).size().reset_index(name='outliers_count')
    
    fig = go.Figure()

    if (display_type == "sensor_id"):
        # Add trace for sensor_id
        fig.add_trace(
          go.Pie(
              labels=outliers_count_per_sensor_per_day["sensor_id"],
              values=outliers_count_per_sensor_per_day["outliers_count"],
              textinfo="label+value+percent",
              name="Sensor ID",
              showlegend=False  # Show legend for this trace
          )
        )
    
    elif (display_type == "date"):
    # Add trace for sensor_id
        fig.add_trace(
          go.Pie(
              labels=outliers_count_per_sensor_per_day["date"],
              values=outliers_count_per_sensor_per_day["outliers_count"],
              textinfo="label+value+percent",
              name="Sensor ID",
              showlegend=False  # Show legend for this trace
          )
        )
            
    
    title_name = "No of incident by " + display_type
    fig.update_layout(title_text=title_name, height=600, width=850)
    fig.show()



# 3. Data Ingestion 

### Read the JSON files from URLs

In [ ]:
# URL of the indoor JSON file
url_indoor = "https://fpardrecruiting002st.z33.web.core.windows.net/CodeEvaluation/DataScientist/iaq.json"

# URL of the outdoor JSON file
url_outdoor = "https://fpardrecruiting002st.z33.web.core.windows.net/CodeEvaluation/DataScientist/oaq.json"

# URL of the Sensor location JSON file
url_sensor_location = "https://fpardrecruiting002st.z33.web.core.windows.net/CodeEvaluation/DataScientist/floors.json"

json_data_indoor = json_api(url_indoor)
json_data_outdoor = json_api(url_outdoor)
json_data_sensor_location = json_api(url_sensor_location)


## 3.1 Convert the Data into pandas Dataframe

In [ ]:
# Convert JSON data to DataFrame
df_indoor = pd.DataFrame(json_data_indoor)
df_outdoor = pd.DataFrame(json_data_outdoor)
df_s_location = pd.DataFrame(json_data_sensor_location)

# 4. Basic Exploration and Data Cleaning

## 4.1 Sensor location

In [ ]:
df_s_location

In [ ]:
df_s_location.info()

There is no missing data. The data has information of the coordinates of each sensor.

### Plotting Sensor location data

In [ ]:
#Storing the index of 3rd floor and 4th floor for access of data in the future
index_H3F = df_s_location[df_s_location['id'] == 'H-3F'].index[0]
index_H4F = df_s_location[df_s_location['id'] == 'H-4F'].index[0]

In [ ]:
# Dictionary to store sensor group information
sensor_group = {}

# Iterate over each row
for index, row in df_s_location.iterrows():

    # Extract the sensors for the current row
    sensors_info = row['sensors']
    
    # Format it into the desired dictionary format
    key = row['id'][2:]  # Extracting '3F' from 'H-3F'
    sensor_group[key] = [sensor['sensor_id'] for sensor in sensors_info]

#### 3rd Floor sensor location plot

In [ ]:
# Plotting the points of 3rd Floor

url = "https://fpardrecruiting002st.z33.web.core.windows.net/CodeEvaluation/DataScientist/H-3F.png"
img = imread(url)

# Get the width and height of the image
width, height = df_s_location['image'][index_H3F]['width'], df_s_location['image'][index_H3F]['height']

# Set the figure size to match the image size
plt.figure(figsize=(width / 100, height / 100))  # Convert pixels to inches
plt.imshow(img)
#plt.axis('off') 

sensors_H3F = df_s_location['sensors'][index_H3F]

# Extract x and y coordinates of sensor locations
sensor_x = [sensor['location']['x'] for sensor in sensors_H3F]
sensor_y = [sensor['location']['y'] for sensor in sensors_H3F]

sensor_ids = [sensor['sensor_id'] for sensor in sensors_H3F]

# Plot sensor locations on top of the image
plt.scatter(sensor_x, sensor_y, color='red', marker='x')

# Add sensor_id labels
for i, txt in enumerate(sensor_ids):
    plt.text(sensor_x[i], sensor_y[i], txt, fontsize=8, color='black', va='bottom', ha='right')

plt.show()

#### 4th Floor sensor location plot

In [ ]:
# Plotting the points of 4th Floor

url = "https://fpardrecruiting002st.z33.web.core.windows.net/CodeEvaluation/DataScientist/H-4F.png"
img = imread(url)

# Get the width and height of the image
width, height = df_s_location['image'][index_H4F]['width'], df_s_location['image'][index_H3F]['height']

# Set the figure size to match the image size
plt.figure(figsize=(width / 100, height / 100))  # Convert pixels to inches
plt.imshow(img)
#plt.axis('off') 

sensors_H4F = df_s_location['sensors'][index_H4F]

# Extract x and y coordinates of sensor locations
sensor_x = [sensor['location']['x'] for sensor in sensors_H4F]
sensor_y = [sensor['location']['y'] for sensor in sensors_H4F]

sensor_ids = [sensor['sensor_id'] for sensor in sensors_H4F]

# Plot sensor locations on top of the image
plt.scatter(sensor_x, sensor_y, color='red', marker='x')

# Add sensor_id labels
for i, txt in enumerate(sensor_ids):
    plt.text(sensor_x[i], sensor_y[i], txt, fontsize=8, color='black', va='bottom', ha='right')

plt.show()

## 4.2 Indoor Data

In [ ]:
df_indoor.info()

The data type of each column seems alright except 'documentTime'. There is no null values, however it could be possible than there are gaps in between 'documentTime'. For example, if we investigate sensor by sensor, we could find some time gaps which are greater than 5 mins as the data is suppose to come every 5 mins.

#### Creating New columns and correcting data type

In [ ]:
#Converting documentTime in datetime format
df_indoor['documentTime'] = pd.to_datetime(df_indoor['documentTime'])

# Sort the DataFrame by 'Timestamp' to ensure timestamps are in order
df_indoor = df_indoor.sort_values(by='documentTime')

#Restting the index and dropping 'index' column
df_indoor = df_indoor.reset_index()
df_indoor = df_indoor.drop(columns={'index'})

Based on the information provided in the documnet, we are creating a two columns i.e. 'temp_c' and 'temp_d'. They will represent temperature at 'ceiling' height and 'desk' heigh respectively. Moreover, a column 'Floor' is also created to map each sensor to its floor which can gives us some insights.

In [ ]:
# Renaming temp to temp_c for temperature at 'ceiling' height
df_indoor = df_indoor.rename(columns={'temp':'temp_c'})

# Renaming temp to temp_d for temperature at 'desk' height
df_indoor['temp_d'] = df_indoor['temp_c']-1.5

# Creting a column Floor' to map each sensor to its floor using sensor_group dictionary
df_indoor['Floor'] = df_indoor['sensor_id'].map({t: floor for floor, types in sensor_group.items() for t in types})

# Extract substring from 'sensor_id' column and concatenate with 'Floor' column
df_indoor['sensor_id_floor_lon'] = df_indoor['sensor_id'] + '_' + df_indoor['Floor']
df_indoor['sensor_id_floor_sho'] = df_indoor['sensor_id'].str[8:] + '_' + df_indoor['Floor']

#### Finding Duplicate rows

In [ ]:
def duplicate_plot(duplicate_rows):
  # Count the occurrences of each sensor type
  duplicate_counts = duplicate_rows['sensor_id_floor_lon'].value_counts().reset_index()
  duplicate_counts.columns = ['sensor_id_floor_lon', 'Count']

  # Plotting the bar chart using Plotly
  fig = px.bar(duplicate_counts, x='sensor_id_floor_lon', y='Count', 
              title='Count of Duplicate Rows for Each Sensor Type',
              labels={'sensor_id_floor_lon': 'Sensor Type', 'Count': 'Count'},
              color='sensor_id_floor_lon')

  fig.show()

In [ ]:
# Find duplicate rows
duplicate_rows = df_indoor[df_indoor.duplicated()]
duplicate_rows

In [ ]:
duplicate_plot(duplicate_rows)

In [ ]:
# Drop duplicate rows
df_indoor = df_indoor.drop_duplicates()

Even though the duplicate columns has been dropped, there could be some cases where for a 'sensor_id' multiple time stamps are available. 

In [ ]:
# Find duplicate rows in 'Time' column for each 'Type'
duplicate_rows = df_indoor[df_indoor.duplicated(subset=['documentTime', 'sensor_id'], keep=False)]

duplicate_rows

#### Finding Gaps in the data based of 'documentTime'

In [ ]:
#removing warnings
pd.options.mode.chained_assignment = None

#Storing all the sensors in sensors list 
sensors = df_indoor['sensor_id_floor_lon'].unique()

#Finding gaps for each sensors
for i in sensors:
    print(i)
    find_gaps(df_indoor[df_indoor['sensor_id_floor_lon']==i],'minutes')
    print("\n")

## 4.2 Outdoor Data

In [ ]:
df_outdoor.info()

### Dealing with duplicate rows

In [ ]:
# Find duplicate rows
duplicate_rows = df_outdoor[df_outdoor.duplicated()]

duplicate_rows

In [ ]:
# Drop duplicate rows
df_outdoor = df_outdoor.drop_duplicates()

In [ ]:
# Find duplicate rows in 'documentTime' column to see if One time stamp has multiple entries.
duplicate_rows = df_outdoor[df_outdoor.duplicated(subset=['documentTime'], keep='first')]

len(duplicate_rows)

We still have 59 cases where there are multiple entries for one timestamp. It could be possible they belong to just the next time stamp. Since there is no clarity now, we'll be dropping those columns.

In [ ]:
#Keeping the first entry of each duplicate value
df_outdoor.drop_duplicates(subset=['documentTime'], keep='first', inplace=True)

### Changing 'documentTime' column to datetime format.

In [ ]:
df_outdoor['documentTime'] = pd.to_datetime(df_outdoor['documentTime'])

# Sort the DataFrame by 'Timestamp' to ensure timestamps are in order
df_outdoor = df_outdoor.sort_values(by='documentTime')

df_outdoor = df_outdoor.reset_index()

df_outdoor = df_outdoor.drop(columns={'index'})

### Look for gaps in 'documentTime'

In [ ]:
find_gaps(df_outdoor,'hours')

In [ ]:
# dropping 'time_diff' column
df_outdoor = df_outdoor.drop(columns={'time_diff'})

In [ ]:
df_outdoor.describe()

### Plotting all the outdoor features in a single plot

In [ ]:
# Plotly line plot
fig = go.Figure()

# 
fig.add_trace(go.Scatter(x=df_outdoor['documentTime'], y=df_outdoor['temperature'], mode='lines', name='temperature'))
fig.add_trace(go.Scatter(x=df_outdoor['documentTime'], y=df_outdoor['humidity'], mode='lines', name='humidity'))
fig.add_trace(go.Scatter(x=df_outdoor['documentTime'], y=df_outdoor['windSpeed'], mode='lines', name='windSpeed'))

# Update layout
fig.update_layout(title='Line Plot of Outdoor temperature and wind speed data',
                  xaxis_title='documentTime',
                  yaxis_title='temp')

# Show plot
fig.show()

Humidity: The overall trend is flat. There have been a couple of dips e.g. Feb 6.

Temperature: It has a sinusoidal sort of pattern. It kept increasing from the start till Feb 3. It started going downward and again started going upwards from 9-10 Feb.

WindSpeed: Most of the data is missing and long flat lines. The data seems to be unreliable to use.

Note: Please select the feature from the legend to analyze one feture at a time.

# 4. How well the Given conditions is maintained

## 4.1. Looking for rows which crosses the given conditions

In [ ]:
# Make a copy of df_indoor dataframe
df_indoor_2= df_indoor.copy()

# Round timestamps to nearest 5 minutes
df_indoor_2['documentTime'] = df_indoor_2['documentTime'].dt.round('5min')

# Set seconds to 0
df_indoor_2['documentTime'] = df_indoor_2['documentTime'].dt.floor('1min')

# Create flags
df_indoor_2['Flag_Temp'] = df_indoor_2['temp_d'] < 21
df_indoor_2['Flag_humidity'] = df_indoor_2['humidity'] < 20
df_indoor_2['Flag_co2'] = df_indoor_2['co2'] > 1000
df_indoor_2['Combined_Flag']= df_indoor_2['Flag_Temp']|df_indoor_2['Flag_humidity']|df_indoor_2['Flag_co2']

In [ ]:
# Create another dataframe for working perios i.e. 9 to 5
df_indoor_2_wh = df_indoor_2.copy()

# Filter rows based on hour component of the timestamp
df_indoor_2_wh = df_indoor_2_wh[(df_indoor_2_wh['documentTime'].dt.hour >= 9) & (df_indoor_2_wh['documentTime'].dt.hour <= 17)]

### All the rows/data points which crosses the given conditions in 3rd Floor

In [ ]:
floor = '3F'
bar_plot_overall(df_indoor_2,floor)
bar_plot_overall(df_indoor_2_wh,floor)

Temperature readings from the sensors contribute the most as far as crossing the limit is concerned in both 'Overall' and 'Working hours'. It is followed by CO2. Interestingly, all the points of CO2 are from assumed 'Working hours' i.e. 9-5. There is no case of Humidity as far as crossing the limit is concerned. Let's find out which sensors and Dates are contributing the most.

### Sensors and Dates where values are crossing the operating conditions in 3rd Floor

In [ ]:
display_type = 'sensor_id'
floor = '3F'
incident_plot(df_indoor_2_wh, display_type, floor)

display_type = 'date'
floor = '3F'
incident_plot(df_indoor_2_wh, display_type, floor)

### All the rows/data points which crosses the given conditions in 4th Floor

In [ ]:
floor = '4F'
bar_plot_overall(df_indoor_2,floor)
bar_plot_overall(df_indoor_2_wh,floor)

### Sensors and Dates where values are crossing the operating conditions in 4th Floor

In [ ]:
display_type = 'sensor_id'
floor = '4F'
incident_plot(df_indoor_2_wh, display_type, floor)

display_type = 'date'
floor = '4F'
incident_plot(df_indoor_2_wh, display_type, floor)

# 5. Data Exploration - Indoor temperature

## 5.1 Combined Floors

### 5.1.1 Statistics of desk temperature for both floors

In [ ]:
# Calculate statistics
statistics = df_indoor.groupby('sensor_id_floor_sho').agg({
    'temp_d': ['mean', 'median', 'max', 'min', 'std']
})

# Columns to store stats values
statistics.columns = ['Mean', 'Median', 'Max', 'Min', 'Std']

# Display as table format
print(statistics)

The 'mean', 'median' and 'Max' seem to similar across all the sensors in 3rd and 4th floor. On the other hand, 'Min' value for sensors indicates some outliers. For example, 'ENKZYW001882_3F' has value as low as 11.5, 'ENKZYW001792_3F' has 15.9 and there are a couple in 17s, 18s and 19s. As far as 'Standard deviation' is concerned, for most of the censors the value revolves around 1 except a few sensor like 'ENKZYW001882_3F' where the value is as high as 1.96.

In [ ]:
# Create subplots
fig = make_subplots(rows=2, cols=2, subplot_titles=('Median','Max','Min Temperature', 'Standard Deviation'))

# Add trace for 'Median' temperature
fig.add_trace(
    go.Bar(x=statistics.index, y=statistics['Median'], name='Median Temperature'),
    row=1, col=1
)

# Add trace for 'Max' temperature
fig.add_trace(
    go.Bar(x=statistics.index, y=statistics['Max'], name='Max Temperature'),
    row=1, col=2
)

# Add trace for 'Min' temperature
fig.add_trace(
    go.Bar(x=statistics.index, y=statistics['Min'], name='Min Temperature'),
    row=2, col=1
)

# Add trace for 'Std'
fig.add_trace(
    go.Bar(x=statistics.index, y=statistics['Std'], name='Standard Deviation'),
    row=2, col=2
)

# Update layout
fig.update_layout(height=700, width=1000, title_text="Temperature Statistics by sensor_id_floor_sho")

#fig.update_layout(xaxis={'tickangle': 45})

# Show plot
fig.show()

### 5.1.2 Trend

In [ ]:
column_name = 'temp_d'
temp_threhold = 21
indoor_combined_single_plot(df_indoor,column_name,temp_threhold)

On looking up all the sensors together including both 3rd-floor and 4th-floor sensors, the overall trend seems to be flat. There are a couple of sensors that are deviating from normal majorly in two instances. One around Jan 29 - Jan 30 and the other around Feb 3 - Feb 4. There are a couple of minor ones, for example, one around Feb 11-13. We'll investigate them further in the later stage.

The plot also shows gaps for the sensors where data is missing e.g. ENKZYW001856_3F and ENKZYW001804_3F.

### 5.1.3 Seasonality

In [ ]:
column_name = 'temp_d'
duration ='time_5'
sesonality(df_indoor_2,column_name,duration)

Daily seasonality can be seen in both the floors.

In [ ]:
duration = 'Date'
sesonality(df_indoor_2,column_name,duration)

For weekly seasonality, the data is quite limited. Still, there are a few observation:

1. 3rd Floor: Not a clear pattern of weekly seasonality.

2. 4th Floor: Some sort of pattern is displayed where the sensors are showing downward trend on weekends

### 5.1.4 Correlation

In [ ]:
column_name = 'temp_d'
correlation_matrix = correlation(df_indoor_2,column_name)

In [ ]:
correlation_plot(correlation_matrix)

In [ ]:
# Display sensors which are strongly correlated to each other
strong_correlation_list(correlation_matrix)

We have used a threshold of 0.5 for analysing the correlation:

Based on the correlation results,

1. Are there any sensors on the 3rd floor that are correlated to sensors on the 4th floor as far as 'desk temperature' is concerned?

In most of the cases, the sesnors of the 3rd floor are not strongly correlated to the sesnors of the 4th floor except in two cases which are '1856_3F' and '1882_3F'. They are positively correlated to '1935_4F' with values more than 0.69. Interestingly, looking at the sensor location, the positions of these sensors are at a similar point (area) on their respective floors.

2. What are the pairs of sensors which are strongly correlated to each other? Are these pairs mounted next (near) to each other?

On the 3rd floor, each sensor is highly correlated to each other except '1856_3F'. It is correlated to '1882_3F' which is mounted near to it. Interestingly, the other sensor i.e. 1848_3F', which seems to be in a similar distance with '1856_3F', is not as highly correlated as '1882_3F'.

On the 4th floor as well, each sensor is highly correlated to each other except 'temp_d_1900_4F'. It is highly correlated to the sensors which are at the right and left of the sensor i.e. temp_d_1925_4F and temp_d_1888_4F.

The correlation can help us to understand the behavior of one sensor concerning others.

Note: For more details, refer to the last output and heat map which are displayed just above.

## 5.2 Indoor Temp - Individual Floors

### 5.2.1 Trend and observations

In [ ]:
column_name = 'temp_d'
temp_threhold = 21
indoor_combined_different_floor_plot(df_indoor,column_name,temp_threhold)

The plots provide more details when looked at separately. Let's discuss the sensors on the 3rd floor first:

Overall it looks alright other than some cases where the values seem to be anomalous. 

1. Feb 2- Feb 3: All the sensors showed a downward trend especially 'ENKZYW001882_3F' which reached around 11°C-13°C and 'ENKZYW001792_3F' which reached around 16-18. All other sensors, whose data is available, also crossed the temperature threshold from the Feb 2 evening to the next day's morning around 10 AM. 

2. Feb 5 (2 AM - 1 PM): All the sensors showed a downward trend and crossed the temperature threshold.

3. Feb 7 - 8: Both 'ENKZYW001882_3F' and 'ENKZYW001848_3F' were out of the threshold most of the time. 'ENKZYW001882_3F' also showed high fluctuation around 11-5 on Feb 7. Interestingly both the sesnors are placed on the other side of the floor.

4. Feb 13 -14:  All the sensors showed an upward trend and some sensors reached their max value e.g. 'ENKZYW001882_3F' touch around 26.

5. Feb 16 -19:  'ENKZYW001856_3F' fell sharply from 25°C to 21°C and remained around 21°C till the end.

All the sensors showed an upward trend and some sensors reached their max value e.g. 'ENKZYW001882_3F' touching around 26.

Let's discuss the sensors on the 4th floor:

The fluctuations of fourth-floor sensors look higher, especially for sensors 'ENKZYW001958_4F' and 'ENKZYW001935_4F'. We can also see some anomalous cases as well:

1. Jan 29 - Jan 30: All the sensors showed a downward trend and crossed the temperature threshold. 

2. Feb 1 - Feb 4: 'ENKZYW001958_4F' was below the threshold in this period and showed high fluctuation.

3. Feb 7: Earning morning, Feb 7 all the sensors went downward, with 'ENKZYW001900_4F' and 'ENKZYW001925_4F' crossing the threshold. Only 'ENKZYW001884_4F' went upwards which is unusual as it highly correlated with sensors '1888_4F', '1883_4F',    '1935_4F', and '1958_4F'.

4. Feb 18 - Feb 19: All the sensors showed a downward trend and some sensors liek'ENKZYW001958_4F','' ENKZYW001883_4F', etc. crossed the threshold.

Let's see if the outdoor temperature is anyhow correlated to any of the indoor temperatures provided by the sensors on the 3rd and 4th floors. It could be possible that it can provide some effect on the sensor values, especially the abnormal behaviors.

### Data Preparation for Indoor temp with outdoor data for both floors together

In [ ]:
df_indoor_3 = df_indoor_2.copy()

# Pivot the DataFrame
pivot_df_indoor_3 = df_indoor_3.pivot_table(index='documentTime', columns='sensor_id_floor_sho', values=['temp_d']).reset_index()

# Flatten the multi-index column names
pivot_df_indoor_3.columns = ['_'.join(col).strip() for col in pivot_df_indoor_3.columns.values]

#Resample it to 1 hour
pivot_df_indoor_3_r = pivot_df_indoor_3.resample('1H', on='documentTime_').first()
pivot_df_indoor_3_r = pivot_df_indoor_3_r.rename(columns={'documentTime_':'documentTime'})

# Merge df1_resampled with df2
merged_df = pd.merge(df_outdoor, pivot_df_indoor_3_r, on='documentTime', how='inner')

#merged_df= merged_df.drop(columns={'level_0','documentTime'})
merged_df_2= merged_df.drop(columns={'documentTime'})

In [ ]:
correlation_matrix = merged_df_2.corr()
correlation_matrix[0:3]

Based on the Matrix, we can say that the Outdoor weather is weekly related to the temperature of indoor for both floors. For outdoor tempearture, the best value is 0.44 with 'temp_d_1848_3F' which is sort of week to moderate. We can also confirm this observation with plotting outdoor parameter with indoor tempearture value.

In [ ]:
# Columns in merged_df
columns_in_out= list(merged_df.columns)
columns_in_out.remove('documentTime')

# Plotly line plot
fig = go.Figure()

# Iterate over each column
for column in columns_in_out:
    fig.add_trace(go.Scatter(x=merged_df['documentTime'], y=merged_df[column], mode='lines', name=column))

# Update layout
fig.update_layout(title= 'Line Plot with all temperature sensor with Outdoor parameters',
                  xaxis_title='documentTime',
                  yaxis_title= column_name)

# Show plot
fig.show()

Though there is one common area where the outdoor temperature is going upwards on the higher side and all the indoor senor temperature data from the 3rd floor is going downward, there is no conclusive evidence as it showed different behavior next time. So, the role of outdoor temperature on indoor data semms not significant.

In [ ]:
# Initialize StandardScaler
scaler = StandardScaler()

# Normalize the columns
df_normalized = pd.DataFrame(scaler.fit_transform(merged_df_2), columns=merged_df_2.columns,index=merged_df_2.index)

df_normalized['documentTime'] = df_outdoor['documentTime']

# Plotly line plot
fig = go.Figure()

# Iterate over each column
for column in columns_in_out:
    fig.add_trace(go.Scatter(x=df_normalized['documentTime'], y=df_normalized[column], mode='lines', name=column))

# Update layout
fig.update_layout(title= 'Normalized Line Plot with all temperature sensor with Outdoor parameters',
                  xaxis_title='documentTime',
                  yaxis_title= column_name)

# Show plot
fig.show()

### 5.2.2 Outliers - 3rd Floor

We have already observed some anomalous bevahious in Trend line plot (Trend Section). This section will give more details in terms of numbers and dates for each sensor.

#### By interquartile range

In [ ]:
# Plotly boxplot
fig = px.box(df_indoor[df_indoor['Floor']=='3F'], x='sensor_id', y='temp_d')
fig.show()

In [ ]:
outliers,outliers_count_per_sensor_per_day=detect_outliers(df_indoor[df_indoor['Floor']=='3F'],'sensor_id','temp_d',1.5,21,'outlier')

sensor_type = "Desk Temperature Outliers"
outlier_plot(outliers_count_per_sensor_per_day, sensor_type)

# Calculate the count of each date
date_outliers_count1 = outliers_count_per_sensor_per_day.groupby(['sensor_id', 'date'])['outliers_count'].sum()
date_outliers_count2 = outliers_count_per_sensor_per_day.groupby(['date', 'sensor_id'])['outliers_count'].sum()
print("-----By Sensor------")
print("\n")
print(date_outliers_count1)
print("\n")
print("-------By Date-----")
print(date_outliers_count2)

#### By "working condition" Threshold

In [ ]:
outliers,outliers_count_per_sensor_per_day=detect_outliers(df_indoor[df_indoor['Floor']=='3F'],'sensor_id','temp_d',1.5,21,'condition')

sensor_type = "Desk Temperature less_than_21"
outlier_plot(outliers_count_per_sensor_per_day, sensor_type)

# Calculate the count of each date
date_outliers_count1 = outliers_count_per_sensor_per_day.groupby(['sensor_id', 'date'])['outliers_count'].sum()
date_outliers_count2 = outliers_count_per_sensor_per_day.groupby(['date', 'sensor_id'])['outliers_count'].sum()
print("-----By Sensor------")
print("\n")
print(date_outliers_count1)
print("\n")
print("-------By Date-----")
print(date_outliers_count2)

### 5.2.3 Outliers - 4th Floor

#### By interquartile range

In [ ]:
# Plotly boxplot
fig = px.box(df_indoor[df_indoor['Floor']=='4F'], x='sensor_id', y='temp_d')
fig.show()

In [ ]:
outliers,outliers_count_per_sensor_per_day=detect_outliers(df_indoor[df_indoor['Floor']=='4F'],'sensor_id','temp_d',1.5,21,'outlier')

sensor_type = "Desk Temperature Outliers"
outlier_plot(outliers_count_per_sensor_per_day, sensor_type)

# Calculate the count of each date
date_outliers_count1 = outliers_count_per_sensor_per_day.groupby(['sensor_id', 'date'])['outliers_count'].sum()
date_outliers_count2 = outliers_count_per_sensor_per_day.groupby(['date', 'sensor_id'])['outliers_count'].sum()
print("-----By Sensor------")
print("\n")
print(date_outliers_count1)
print("\n")
print("-------By Date-----")
print(date_outliers_count2)

#### By "working condition" Threshold

In [ ]:
outliers,outliers_count_per_sensor_per_day=detect_outliers(df_indoor[df_indoor['Floor']=='4F'],'sensor_id','temp_d',1.5,21,'condition')

sensor_type = "Desk Temperature less_than_21"
outlier_plot(outliers_count_per_sensor_per_day, sensor_type)

# Calculate the count of each date
date_outliers_count1 = outliers_count_per_sensor_per_day.groupby(['sensor_id', 'date'])['outliers_count'].sum()
date_outliers_count2 = outliers_count_per_sensor_per_day.groupby(['date', 'sensor_id'])['outliers_count'].sum()
print("-----By Sensor------")
print("\n")
print(date_outliers_count1)
print("\n")
print("-------By Date-----")
print(date_outliers_count2)

#### Observation

If we look at observing the outliers calculated using IQR or Box plot for both floors, the count for common dates is quite less on comparing the 3rd floor to the 4th floor. It suggests that the incidents which have been observed are not connected. It also corroborates the fact that most of the sensors on the third floor are not correlated to the sensors on the fourth floor.

In the next sections i.e. Humidity and CO2, we'll also try to find if there are in any similarities in the abnormal values as per indoor temperature data.

# 6. Data Exploration - Humidity

## 6.1 Combined Floors

### 6.1.1 Statistics of Humidity for both floors

In [ ]:
# Calculate statistics
statistics = df_indoor.groupby('sensor_id_floor_sho').agg({
    'humidity': ['mean', 'median', 'max', 'min', 'std']
})

# Columns to store ststs values
statistics.columns = ['Mean', 'Median', 'Max', 'Min', 'Std']

# Display as table format
print(statistics)

In [ ]:
# Create subplots
fig = make_subplots(rows=2, cols=2, subplot_titles=('Median','Max','Min', 'Standard Deviation'))

# Add trace for 'Median' temperature
fig.add_trace(
    go.Bar(x=statistics.index, y=statistics['Median'], name='Median Humidity'),
    row=1, col=1
)

# Add trace for 'Max' temperature
fig.add_trace(
    go.Bar(x=statistics.index, y=statistics['Max'], name='Max Humidity'),
    row=1, col=2
)

# Add trace for 'Min' temperature
fig.add_trace(
    go.Bar(x=statistics.index, y=statistics['Min'], name='Min Humidity'),
    row=2, col=1
)

# Add trace for 'Std'
fig.add_trace(
    go.Bar(x=statistics.index, y=statistics['Std'], name='Standard Deviation'),
    row=2, col=2
)

# Update layout
fig.update_layout(height=700, width=1000, title_text="Humidity Statistics by sensor_id_floor_sho")

#fig.update_layout(xaxis={'tickangle': 45})

# Show plot
fig.show()

### 6.1.2 Trend

In [ ]:
# Plotly line plot
fig = go.Figure()

# Iterate over each unique type and plot its line
for type_val in df_indoor['sensor_id_floor_lon'].unique():
    df_type = df_indoor[(df_indoor['sensor_id_floor_lon'] == type_val)]
    fig.add_trace(go.Scatter(x=df_type['documentTime'], y=df_type['humidity'], mode='lines', name=type_val))

# Update layout
fig.update_layout(title='Line Plot of Temperature by Sensor',
                  xaxis_title='documentTime',
                  yaxis_title='humidity')

# Show plot
fig.show()

On looking up all the sensors together including both 3rd-floor and 4th-floor sensors, the overall trend seems to be flat. However in the first half (before Feb 5) and From Feb 10 - Feb 19, there is a increasing trend.

The sensors looks highly correlated to each other. It will be confirmed using correlation matrix.

The plot also shows gaps for the sensors where data is missing e.g. ENKZYW001856_3F and ENKZYW001804_3F.

There are a couple of sensors that are deviating from normal majorly around Feb 2 - Feb 4. It will be investigated later.

### 6.1.3 Seasonality

In [ ]:
column_name = 'humidity'
duration ='time_5'
sesonality(df_indoor_2,column_name,duration)

Daily seasonality can be seen in both the floors. On 4th Floor, it looks a bit stronger.

In [ ]:
duration = 'Date'
sesonality(df_indoor_2,column_name,duration)

For weekly seasnality, the data is quite limited. On both the floors, not a clear pattern of weekly seasonality.

### 6.1.4 Correlation

In [ ]:
correlation_matrix = correlation(df_indoor_2,column_name)

In [ ]:
correlation_plot(correlation_matrix)

In [ ]:
# Display sensors which are strongly correlated to each other
strong_correlation_list(correlation_matrix)

We have used threshold of 0.5 for analysing the correlation:

Based on the correlation results, we can clearly see that all the sensors are strongly correlated to each other as far as 'humidity' is concerned except 1856_3F. One can thought, it could be possible due to its location but on the 4th floor, 1935_4F is also highly correlated with almost all the sensors.

## 6.2 Humidity - Individual Floors

### 6.2.1 Trend and observations

In [ ]:
column_name = 'humidity'
temp_threhold = 20
indoor_combined_different_floor_plot(df_indoor,column_name,temp_threhold)

The plots provide more details when looked at separtly. There is no instance of a value crossing the given threshold.

Based on the plots the only anomalous reason seems to be in the third floor around the Feb2 - Feb4 period. All the values are going in upward direction with some sensors going really high e.g. 1882_3F and '1792_3F'. 

Let's see if the outdoor temperature is anyhow correlated to any of the indoor temperatures provided by the sensors on the 3rd and 4th floors. It could be possible that it can provide some effect on the sensor values, especially the abnormal behaviors.

### Data Preparation for Humidity with outdoor data for both floors together

In [ ]:
df_indoor_3 = df_indoor_2.copy()

# Pivot the DataFrame
pivot_df_indoor_3 = df_indoor_3.pivot_table(index='documentTime', columns='sensor_id_floor_sho', values=['humidity']).reset_index()

# Flatten the multi-index column names
pivot_df_indoor_3.columns = ['_'.join(col).strip() for col in pivot_df_indoor_3.columns.values]

#Resample it to 1 hour
pivot_df_indoor_3_r = pivot_df_indoor_3.resample('1H', on='documentTime_').first()
pivot_df_indoor_3_r = pivot_df_indoor_3_r.rename(columns={'documentTime_':'documentTime'})

# Merge df1_resampled with df2
merged_df = pd.merge(df_outdoor, pivot_df_indoor_3_r, on='documentTime', how='inner')

#merged_df= merged_df.drop(columns={'level_0','documentTime'})
merged_df_2= merged_df.drop(columns={'documentTime'})

In [ ]:
correlation_matrix = merged_df_2.corr()
correlation_matrix[0:3]

The external temperature seems to be strongly correlated to all the sensors except 1856_3F as far as indoor 'humidity' values are concerned. On the other hand, external humidity is not at all correlated. We can also confirm this observation with plotting outdoor parameter with indoor humidity value.

In [ ]:
# Columns in merged_df
columns_in_out= list(merged_df.columns)
columns_in_out.remove('documentTime')

# Plotly line plot
fig = go.Figure()

# Iterate over each column
for column in columns_in_out:
    fig.add_trace(go.Scatter(x=merged_df['documentTime'], y=merged_df[column], mode='lines', name=column))

# Update layout
fig.update_layout(title= 'Line Plot with all Humidity sensor with Outdoor parameters',
                  xaxis_title='documentTime',
                  yaxis_title= column_name)

# Show plot
fig.show()

In [ ]:
# Initialize StandardScaler
scaler = StandardScaler()

# Normalize the columns
df_normalized = pd.DataFrame(scaler.fit_transform(merged_df_2), columns=merged_df_2.columns,index=merged_df_2.index)

df_normalized['documentTime'] = df_outdoor['documentTime']

# Plotly line plot
fig = go.Figure()

# Iterate over each column
for column in columns_in_out:
    fig.add_trace(go.Scatter(x=df_normalized['documentTime'], y=df_normalized[column], mode='lines', name=column))

# Update layout
fig.update_layout(title= 'Normalized Line Plot with all Humidity sensor with Outdoor parameters',
                  xaxis_title='documentTime',
                  yaxis_title= column_name)

# Show plot
fig.show()

It can be seen that when the temperature goes up the humidity value also goes up and the anomalous region on the third floor around the Feb2 - Feb4 period could be related to the rise of temperature.

### 6.2.2 Outliers Humidity - 3rd Floor

We have already observed some anomalous bevahious in Trend line plot (Trend Section). This section will give more details in terms of numbers and dates for each sensor.

#### By interquartile range

In [ ]:
# Plotly boxplot
fig = px.box(df_indoor[df_indoor['Floor']=='3F'], x='sensor_id', y='humidity')
fig.show()

In [ ]:
outliers,outliers_count_per_sensor_per_day=detect_outliers(df_indoor[df_indoor['Floor']=='3F'],'sensor_id','humidity',1.5,20,'outlier')

sensor_type = "Humidity Outliers"
outlier_plot(outliers_count_per_sensor_per_day, sensor_type)

# Calculate the count of each date
date_outliers_count1 = outliers_count_per_sensor_per_day.groupby(['sensor_id', 'date'])['outliers_count'].sum()
date_outliers_count2 = outliers_count_per_sensor_per_day.groupby(['date', 'sensor_id'])['outliers_count'].sum()
print("-----By Sensor------")
print("\n")
print(date_outliers_count1)
print("\n")
print("-------By Date-----")
print(date_outliers_count2)

Since we have no case for values crossing condition threshold, we'll not be plotting "Outlier By 'working condition' Threshold".

### 6.2.3 Outliers Humidity - 4th Floor

In [ ]:
# Plotly boxplot
fig = px.box(df_indoor[df_indoor['Floor']=='4F'], x='sensor_id', y='humidity')
fig.show()

There is no outlier displayed in this cases.

#### Observation/Comments

We observed this on the 3rd floor for temperature values:

Feb 2- Feb 3: All the sensors showed downward trend especially 'ENKZYW001882_3F' which reached around 11°C-13°C and 'ENKZYW001792_3F' which reached around 16-18. All other sensors, whose data is available, also crossed the temperature threshold from Feb 2 evening to next day morning around 10 AM. 

And if I look at humidity plot for 3rd floor:
 
Around Feb2 - Feb4 period. All the values are going in upward direction with some sensors going really high e.g. 'ENKZYW001882_3F and 'ENKZYW001792_3F'.

So, it is highly likely that there is connection between these two.

# 7. Data Exploration - Co2

## 7.1 Combined Floors

### 7.1.1 Statistics of Co2 for both floors

In [ ]:
# Calculate statistics
statistics = df_indoor.groupby('sensor_id_floor_sho').agg({
    'co2': ['mean', 'median', 'max', 'min', 'std']
})

# Columns to store ststs values
statistics.columns = ['Mean', 'Median', 'Max', 'Min', 'Std']

# Display as table format
print(statistics)

In [ ]:
# Create subplots
fig = make_subplots(rows=2, cols=2, subplot_titles=('Median','Max','Min', 'Standard Deviation'))

# Add trace for 'Median' temperature
fig.add_trace(
    go.Bar(x=statistics.index, y=statistics['Median'], name='Median Humidity'),
    row=1, col=1
)

# Add trace for 'Max' temperature
fig.add_trace(
    go.Bar(x=statistics.index, y=statistics['Max'], name='Max Humidity'),
    row=1, col=2
)

# Add trace for 'Min' temperature
fig.add_trace(
    go.Bar(x=statistics.index, y=statistics['Min'], name='Min Humidity'),
    row=2, col=1
)

# Add trace for 'Std'
fig.add_trace(
    go.Bar(x=statistics.index, y=statistics['Std'], name='Standard Deviation'),
    row=2, col=2
)

# Update layout
fig.update_layout(height=700, width=1000, title_text="Humidity Statistics by sensor_id_floor_sho")

#fig.update_layout(xaxis={'tickangle': 45})

# Show plot
fig.show()

### 7.1.2 Trend

In [ ]:
# Plotly line plot
fig = go.Figure()

# Iterate over each unique type and plot its line
for type_val in df_indoor['sensor_id_floor_lon'].unique():
    df_type = df_indoor[(df_indoor['sensor_id_floor_lon'] == type_val)]
    fig.add_trace(go.Scatter(x=df_type['documentTime'], y=df_type['co2'], mode='lines', name=type_val))
    
# Add a flat dotted line of y = 1000
fig.add_trace(go.Scatter(x=[min(df_indoor['documentTime']), max(df_indoor['documentTime'])], y=[1000, 1000],
                         mode='lines', name='y=100', line=dict(dash='dot', color='black', width=2)))

# Update layout
fig.update_layout(title='Line Plot of Temperature by Sensor',
                  xaxis_title='documentTime',
                  yaxis_title='co2')

# Show plot
fig.show()

On looking up all the sensors together including both 3rd-floor and 4th-floor sensors, the overall trend seems to be quite flactuating like sine wave unlike indoor and humidity plots. It is indicating some seasonality.

The sensors looks highly correlated to each other. It will be confirmed using correlation matrix.

The plot also shows gaps for the sensors where data is missing e.g. ENKZYW001856_3F and ENKZYW001804_3F.

There are a couple of sensors that are crossing condition threshold. It will be investigated later.

### 7.1.3 Seasonality

In [ ]:
column_name = 'co2'
duration ='time_5'
sesonality(df_indoor_2,column_name,duration)

As expected, Co2 is showing higher daily seasonality for both the floors.

In [ ]:
duration = 'Date'
sesonality(df_indoor_2,column_name,duration)

As expected, Co2 is showing higher weekly seasonality for both the floors.

### 7.1.4 Correlation

In [ ]:
correlation_matrix = correlation(df_indoor_2,column_name)

In [ ]:
correlation_plot(correlation_matrix)

In [ ]:
# Display sensors which are strongly correlated to each other
strong_correlation_list(correlation_matrix)

We have used threshold of 0.5 for analysing the correlation:

As expected, Based on the correlation results, we can clearly see that all the sensors are strongly correlated to each other as far as 'Co2' is concerned except 1856_3F.

## 7.2 Co2 - Individual Floors

### 7.2.1 Trend and observations

In [ ]:
column_name = 'co2'
temp_threhold = 1000
indoor_combined_different_floor_plot(df_indoor,column_name,temp_threhold)

The plots provide more details when looked at separately. 

Based on the plots the main anomalous reason seems to be around January 29th when the trend of all the sensors on the 3rd floor looks quite identical to the sensors on the 4th floor. The values are going up in the morning of the 29th around 8-9 AM and crossing the threshold to reach quite a high value of 1200-1300. It started going downward around 2-3 PM.

Apart from this, third-floor behavior seems reasonable. However, on the fourth floor, there are a couple of incidents where values are going above the threshold. For example, around Feb 7 and Feb 8, 1935_4, F1900_4F and 1888_4F are crossing the threshold.


Let's see if the outdoor variables are anyhow correlated to any of the co2 values provided by the sensors on the 3rd and 4th floors. It could be possible that it can have some effect on the sensor values, especially the abnormal behaviors.

In [ ]:
### Data Preparation of Co2 with outdoor data for both floors together

In [ ]:
df_indoor_3 = df_indoor_2.copy()

# Pivot the DataFrame
pivot_df_indoor_3 = df_indoor_3.pivot_table(index='documentTime', columns='sensor_id_floor_sho', values=['co2']).reset_index()

# Flatten the multi-index column names
pivot_df_indoor_3.columns = ['_'.join(col).strip() for col in pivot_df_indoor_3.columns.values]

#Resample it to 1 hour
pivot_df_indoor_3_r = pivot_df_indoor_3.resample('1H', on='documentTime_').first()
pivot_df_indoor_3_r = pivot_df_indoor_3_r.rename(columns={'documentTime_':'documentTime'})

# Merge df1_resampled with df2
merged_df = pd.merge(df_outdoor, pivot_df_indoor_3_r, on='documentTime', how='inner')

#merged_df= merged_df.drop(columns={'level_0','documentTime'})
merged_df_2= merged_df.drop(columns={'documentTime'})

In [ ]:
correlation_matrix = merged_df_2.corr()
correlation_matrix[0:3]

Based on the Matrix, we can say that the Outdoor weather is weekly related to the co2 of indoor for both floors. We can also confirm this observation with plotting outdoor parameter with indoor tempearture value.

In [ ]:
# Columns in merged_df
columns_in_out= list(merged_df.columns)
columns_in_out.remove('documentTime')

# Plotly line plot
fig = go.Figure()

# Iterate over each column
for column in columns_in_out:
    fig.add_trace(go.Scatter(x=merged_df['documentTime'], y=merged_df[column], mode='lines', name=column))

# Update layout
fig.update_layout(title= 'Line Plot with all Humidity sensor with Outdoor parameters',
                  xaxis_title='documentTime',
                  yaxis_title= column_name)

# Show plot
fig.show()

This plot is not giving much idea due to scale difference.

In [ ]:
# Initialize StandardScaler
scaler = StandardScaler()

# Normalize the columns
df_normalized = pd.DataFrame(scaler.fit_transform(merged_df_2), columns=merged_df_2.columns,index=merged_df_2.index)

df_normalized['documentTime'] = df_outdoor['documentTime']

# Plotly line plot
fig = go.Figure()

# Iterate over each column
for column in columns_in_out:
    fig.add_trace(go.Scatter(x=df_normalized['documentTime'], y=df_normalized[column], mode='lines', name=column))

# Update layout
fig.update_layout(title= 'Normalized Line Plot with all Humidity sensor with Outdoor parameters',
                  xaxis_title='documentTime',
                  yaxis_title= column_name)

# Show plot
fig.show()

If we look at the anomalous region of January 29th when the values went up crossing the threshold to reach quite a high value of 1200-1300, the outdoor temperature is going upwards from the lowest ever higher side. However, the pattern looks not be repeating. So, the role of outdoor temperature on indoor data seems not significant.

### 7.2.2 Outliers Co2 - 3rd Floor

#### By interquartile range

In [ ]:
# Plotly boxplot
fig = px.box(df_indoor[df_indoor['Floor']=='3F'], x='sensor_id', y='co2')
fig.show()

In [ ]:
outliers,outliers_count_per_sensor_per_day=detect_outliers(df_indoor[df_indoor['Floor']=='3F'],'sensor_id','co2',1.5,1000,'outlier')

sensor_type = "Co2 Outliers"
outlier_plot(outliers_count_per_sensor_per_day, sensor_type)

# Calculate the count of each date
date_outliers_count1 = outliers_count_per_sensor_per_day.groupby(['sensor_id', 'date'])['outliers_count'].sum()
date_outliers_count2 = outliers_count_per_sensor_per_day.groupby(['date', 'sensor_id'])['outliers_count'].sum()
print("-----By Sensor------")
print("\n")
print(date_outliers_count1)
print("\n")
print("-------By Date-----")
print(date_outliers_count2)

#### By "working condition" Threshold

In [ ]:
outliers,outliers_count_per_sensor_per_day=detect_outliers(df_indoor[df_indoor['Floor']=='3F'],'sensor_id','co2',1.5,1000,'condition')

sensor_type = "Co2 Outliers"
outlier_plot(outliers_count_per_sensor_per_day, sensor_type)

# Calculate the count of each date
date_outliers_count1 = outliers_count_per_sensor_per_day.groupby(['sensor_id', 'date'])['outliers_count'].sum()
date_outliers_count2 = outliers_count_per_sensor_per_day.groupby(['date', 'sensor_id'])['outliers_count'].sum()
print("-----By Sensor------")
print("\n")
print(date_outliers_count1)
print("\n")
print("-------By Date-----")
print(date_outliers_count2)

### 7.2.3 Outliers Co2 - 4th Floor

#### By interquartile range

In [ ]:
# Plotly boxplot
fig = px.box(df_indoor[df_indoor['Floor']=='4F'], x='sensor_id', y='co2')
fig.show()

In [ ]:
outliers,outliers_count_per_sensor_per_day=detect_outliers(df_indoor[df_indoor['Floor']=='4F'],'sensor_id','co2',1.5,1000,'outlier')

sensor_type = "Co2 Outliers"
outlier_plot(outliers_count_per_sensor_per_day, sensor_type)

# Calculate the count of each date
date_outliers_count1 = outliers_count_per_sensor_per_day.groupby(['sensor_id', 'date'])['outliers_count'].sum()
date_outliers_count2 = outliers_count_per_sensor_per_day.groupby(['date', 'sensor_id'])['outliers_count'].sum()
print("-----By Sensor------")
print("\n")
print(date_outliers_count1)
print("\n")
print("-------By Date-----")
print(date_outliers_count2)

#### By "working condition" Threshold

In [ ]:
outliers,outliers_count_per_sensor_per_day=detect_outliers(df_indoor[df_indoor['Floor']=='4F'],'sensor_id','co2',1.5,1000,'condition')

sensor_type = "Co2 Outliers"
outlier_plot(outliers_count_per_sensor_per_day, sensor_type)

# Calculate the count of each date
date_outliers_count1 = outliers_count_per_sensor_per_day.groupby(['sensor_id', 'date'])['outliers_count'].sum()
date_outliers_count2 = outliers_count_per_sensor_per_day.groupby(['date', 'sensor_id'])['outliers_count'].sum()
print("-----By Sensor------")
print("\n")
print(date_outliers_count1)
print("\n")
print("-------By Date-----")
print(date_outliers_count2)